In [1]:
!pip install numpy
!pip install matplotlib
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -------------------- ------------------- 6.6/12.8 MB 34.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.8 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 23.5 MB/s  0:00:00
Defaulting to user installation because normal site-packages is not writeable
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------- ------------------------------ 1.8/8.1 MB 9.9 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 14.4 MB/s  0:00:00
Using cached contourpy-1.3.3-cp

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
class AntColony:
   def __init__(self, sizes, capacity, n_ants, n_best, n_iterations, decay, alpha, beta, R_max):

    self.sizes = sizes
    self.capacity = capacity
    self.n = len(sizes)

    self.n_ants = n_ants
    self.n_best = n_best
    self.n_iterations = n_iterations
    self.decay = decay
    self.alpha = alpha
    self.beta = beta
    self.R_max = R_max

    # Pheromone per rank (0 = open new bin)
    self.pheromone = np.ones(R_max + 1)

    # Sort items largest to smallest
    self.order = sorted(range(self.n), key=lambda i: -self.sizes[i])

    # --- Helper functions ---

   def heuristic(self, residual):
      # Chooses the bin with the smallest leftover space
      return 1.0 / (1.0 + residual)

   def choice_probabilities(self, feasible_bins, new_residual):
      # Compute seleciton probability for each bin (existing bin + new bin)
      options = []

      #Existing bins (rank 1..R_max)
      for rank, residual in feasible_bins:
        limited_rank = min(rank, self.R_max)
        Pheromone_strength = self.pheromone[limited_rank] ** self.alpha
        heuristic_value = self.heuristic(residual) ** self.beta
        options.append((limited_rank, Pheromone_strength * heuristic_value))

      #New bin (rank 0)
      Pheromone_new = self.pheromone[0] ** self.alpha
      heuristic_new = self.heuristic(new_residual) ** self.beta
      options.append((0, Pheromone_new * heuristic_new))

      #Normalizie selection probabilities
      total = sum(score for _, score in options)
      probabilities = [(limited_rank, score / total) for limited_rank, score in options]
      return probabilities

   def sample_rank(self, probabilities):
      # Randomly choose a rank based on its probability
      random_value = np.random.random()
      running_total = 0.0
      for rank, prob in probabilities:
        running_total += prob
        if random_value <= running_total:
          return rank
      return probabilities